In [1]:
from tabulate import tabulate
import math

In [2]:
def CKYParse(words, grammar, table):
  for j in range(0, len(words)):
    for A in grammar[words[j]]: #get all a-> word[j]
      table[j][j] = table [j][j].union([A])
    for i in range (j, 0, -1):
      #print (i-1, j)
      for k in range(i-1, j):
        for A in grammar["non-terminal"]:
          #print (A[1], table[i-1][k], A[2], table[k+1][j])
          if A[1] in table[i-1][k] and A[2] in table[k+1][j]:
            table[i-1][j] = table[i-1][j].union([A[0]])
          #print (tabulate(table))
  return table

#part 3

In [33]:
class Node():
    """A node in a parse tree
    """

    def __init__(self, tag, str_content = None , children = []):
        """Creates a new node.

        Args:
            tag (str): The POS tag of the current node.
            str_content (str, optional): . The content of the current node. Either this must be set or children. Defaults to None.
            children (List[Node], optional): The children of the current node. Either this must be set or str_content. Defaults to [].
        """
        self.tag = tag
        if str_content:
            self.children = [str_content] + children
        else:
            self.children = children

    def __str__(self):
        return f"[{self.tag} " + " ".join([str(c) for c in self.children]) + "]"

    def __repr__(self):
        return str(self)

class CKYTable():

    def __init__(self, size, use_dict = False):
        """Creates a new table for CKY parsing

        Args:
            size (int): The size of the parsing table that is required
            use_dict (bool, optional): if to use set or dictionary cells. Defaults to True.
        """
        #self.table = [[{} if use_dict else set() for i in range(j + 1)] for j in range(size)]
        self.table = [[set() for j in range (len(words))] for i in range(len(words))]

    def __getitem__(self, key):
        if isinstance(key, str) or isinstance(key, int):
            return self.table[key]
        else:
            cell = self.table
            for keyComponent in key:
                cell = cell[keyComponent]
            return cell

    def __setitem__(self, key, value):
        if isinstance(key, str) or isinstance(key, int):
            self.table[key] = value
        else:
            cell = self.table
            for i, keyComponent in enumerate(key):
                if i == len(key) - 1:
                    cell[keyComponent] = value
                else:
                    cell = cell[keyComponent]

    def __len__(self):
        return len(self.table)

    def __repr__(self):
        return tabulate(self.table)

    def __str__(self):
        return tabulate(self.table)

class CKYResult():

    def __init__(self, prob_table, words, backptr_table = None):
        """Stores the results of a CKY probabilistic parse.

        Args:
            prob_table (CKYTable): The resulting probabilities table from the parse
            words (List[str]): The sentence tht was being parsed
            backptr_table (Optional[CKYTable], optional): The table of backpointers to use t ore-create the most probable parsed graph. Defaults to None.
        """
        self.prob_table = prob_table
        self.backptr_table = backptr_table
        self.words = words

    def get_most_probable_root(self):
        """Gets the most probable root POS tag with its probability.

        Returns:
            Dict[str, Union[str, float]]: Dictionary containing the most probable root POS tag and its probability
        """
        root_tag, root_prob =  max(self.prob_table[(len(self.words) - 1, 0)].items(), key = lambda x: x[1])
        return {'tag': root_tag, 'prob': root_prob}

    def to_graph(self):
        """Gets the graph representation of the parse table. Requires backpointers table to be initialised

        Returns:
            Optional[Node]: THe root node of the tree, or None
        """
        if len(self.prob_table[(len(self.words) - 1, 0)]) > 0 and self.backptr_table is not None:
            max_parse = self.get_most_probable_root()
            return self._construct_graph_helper((len(self.words) - 1, 0, max_parse['tag']))
        else: 
            return None

    def __str__(self):
        return tabulate(self.prob_table.table,  headers=self.words)

    def __repr__(self):
        return tabulate(self.prob_table.table,  headers=self.words)

    def _construct_graph_helper(self, ptr) :
        """Given a pointer to a starting position in the table, constructs a tree for that position.

        Args:
            ptr (Tuple[int, int, str]): The root of the tree to d   ecode

        Returns:
            Node: The deocded root of the tree
        """
        if self.backptr_table[ptr] is None:
            return Node(ptr[2], self.words[ptr[0]])
        else:
            backptr = self.backptr_table[ptr]
            return Node(ptr[2], children = [self._construct_graph_helper(backptr[0]), self._construct_graph_helper(backptr[1])])
'''
def cky_prob_parse(words: List[str], rules: Dict[Tuple[str, str], Dict[str, float]], terminal_rules: Dict[str, Dict[str, float]]) -> CKYResult:
    """Calculates the CKY table for a probabilistic context free grammar for a sentence.

    Args:
        words (List[str]): The tokenised sentence to run the CKY algorithm over
        rules (Dict[Tuple[str, str], Tuple[str, float]]): The nonterminal rules present in the grammar with their probabilities
        terminal_rules (Dict[str, Dict[str, float]]): The terminal rules present in the grammar with their probabilities.

    Returns:
        CKYResult: The resulting parse.
    """
    # Use column, row notation
    table = CKYTable(len(words))
    backpointers = CKYTable(len(words))
    # Generate starting non-terminals
    for col in range(len(words)):
        table[(col, col)].update(terminal_rules[words[col]])
        backpointers[(col, col)].update(terminal_rules[words[col]])
        
        # Set probability as log of probability, set backpointers to NULL
        for k in table[(col, col)].keys():
            table[(col, col, k)] = math.log(table[(col, col, k)])
        # Start constructing possible combinations
        for row in range(col - 1, -1, -1): # Is exclusive upperbound, so use -1
            for peek_idx in range(row, col):
                for combination in combinations(table[peek_idx][row].keys(), table[col][peek_idx + 1].keys()):
                    if combination in rules:
                        
                        # Calculate probability in log space and indexes
                        idx0 = (peek_idx, row, combination[0])
                        idx1 = (col, peek_idx + 1, combination[1])
                        for comb_tag, comb_prob in rules[combination].items():
                            current_cell_idx = (col, row, comb_tag)
                            combination_prob = math.log(comb_prob) + table[idx0] + table[idx1]
                            if comb_tag not in table[col][row] or table[current_cell_idx] <  combination_prob:
                                table[current_cell_idx] = combination_prob
                                backpointers[current_cell_idx] = (idx0, idx1)
    return CKYResult(table, words, backpointers)'''

'\ndef cky_prob_parse(words: List[str], rules: Dict[Tuple[str, str], Dict[str, float]], terminal_rules: Dict[str, Dict[str, float]]) -> CKYResult:\n    """Calculates the CKY table for a probabilistic context free grammar for a sentence.\n\n    Args:\n        words (List[str]): The tokenised sentence to run the CKY algorithm over\n        rules (Dict[Tuple[str, str], Tuple[str, float]]): The nonterminal rules present in the grammar with their probabilities\n        terminal_rules (Dict[str, Dict[str, float]]): The terminal rules present in the grammar with their probabilities.\n\n    Returns:\n        CKYResult: The resulting parse.\n    """\n    # Use column, row notation\n    table = CKYTable(len(words))\n    backpointers = CKYTable(len(words))\n    # Generate starting non-terminals\n    for col in range(len(words)):\n        table[(col, col)].update(terminal_rules[words[col]])\n        backpointers[(col, col)].update(terminal_rules[words[col]])\n        \n        # Set probability as

In [18]:
def CKYParseWeighted(words, grammar, table):
  tree = []
  probability = []
  for j in range(0, len(words)):
    for A in grammar[words[j]]: #get all a-> word[j]
      table[j][j] = table [j][j].union([A[0]])
    for i in range (j, 0, -1):
      #print (i-1, j)
      for k in range(i-1, j):
        for A in grammar["non-terminal"]:
          #print (A[1], table[i-1][k], A[2], table[k+1][j])
          if A[1] in table[i-1][k] and A[2] in table[k+1][j]:
            table[i-1][j] = table[i-1][j].union([A[0]])
          #print (tabulate(table))
  return table, tree, probability

#part 4

In [48]:
from functools import reduce
def combinations(set1, set2):
    """Calculates all possible combinations of values from 2 sets keeping order

    Args:
        set1 (Iterable): Set 1 to iterate over
        set2 (Iterable): Set 2 to iteratae over

    Returns:
        List[Tuple]: List of all possible combinations
    """
    return [(s1, s2) for s1 in set1 for s2 in set2]
def cky_prob_marginalise_parse(words, grammar):
    """Computes the marginal over a PCFG using CKY parsing

    Args:
        words (Iterable[str]): The sentence to get the probability of
        rules (Dict[Tuple[str, str], Tuple[str, float]]): The rules for non-terminals
        terminal_rules (Dict[str, Dict[str, float]]): The rules for terminals

    Returns:
        CKYResult: The resulting parse of the sentence using the PCFG grammar
    """
    # Use column, row notation
    table = CKYTable(len(words))
    # Generate starting non-terminals
    for j in range(0, len(words)):
      for A in grammar[words[j]]:
        table[(j, j)] = table[(j, j)].union([A])
        
        # Set probability as log of probability, set backpointers to NULL
        #print (table[(j, j)])
        for k in table[(j, j)]:
            #check this
            print (k)
            k = (k[0], math.log(k[1]))
            print (k)
        # Start constructing possible combinations
  #for j in range(0, len(words)):
    #for A in grammar[words[j]]: #get all a-> word[j]
      #table[j][j] = table [j][j].union([A])
    #for i in range (j, 0, -1):
      #print (i-1, j)
     ## for k in range(i-1, j):
       # for A in grammar["non-terminal"]:
        #  if A[1] in table[i-1][k] and A[2] in table[k+1][j]:
        #    table[i-1][j] = table[i-1][j].union([A[0]])
            

        for row in range(j - 1, -1, -1): # Is exclusive upperbound, so use -1
            for peek_idx in range(row, j):
                for combination in combinations(table[peek_idx][row], table[j][peek_idx + 1]):
                    print (combination)
                    if combination in grammar["non-terminal"]:
                        # Calculate probability in log space and indexes
                        idx0 = (peek_idx, row, combination[0])
                        idx1 = (j, peek_idx + 1, combination[1])
                        for comb_tag, comb_prob in grammar["non-terminal"][combination].items():
                            current_cell_idx = (j, row, comb_tag)
                            combination_prob = math.log(comb_prob) + table[idx0] + table[idx1]
                            if comb_tag not in table[j][row]:
                                table[current_cell_idx] = combination_prob
                            else:
                                # Use addition in log space from: https://en.wikipedia.org/wiki/Log_probability
                                table[current_cell_idx] += math.log1p(math.exp(combination_prob - table[current_cell_idx]))
    return CKYResult(table, words)
def log_add(x, y):
    x + math.log1p(math.exp(y - x))

words = ["astronomers", "saw", "stars", "with", "ears"]
grammar = {"non-terminal": [("S", "NP", "VP", 1.0), ("NP", "NP", "PP", 0.4), ("PP", "P", "NP", 1.0),("VP", "V", "NP", 0.7), ("VP", "VP", "PP", 0.3)],
                            "astronomers": [("NP", 0.4)] ,  "ears": [("NP", 0.18)], "saw": [("NP", 0.04), ("V", 1.0)], "with": [("P", 1.0)], 
                            "stars": [("NP", 0.18)], "telescopes": [("NP", 0.1)]}
result = cky_prob_marginalise_parse(words, grammar)
print("Marginalised parse table:")
print(result)
sum_prob = reduce(log_add, result.prob_table[(len(result.words) - 1, 0)].values())
print(f"Marginalised probability: {math.exp(sum_prob)}, log probability: {sum_prob}")

('NP', 0.4)
('NP', 0.4)
('NP', 0.04)
('NP', 0.04)
(('NP', -0.916290731874155), ('NP', -3.2188758248682006))
(('NP', -0.916290731874155), ('NP', 0.04))
(('NP', 0.4), ('NP', -3.2188758248682006))
(('NP', 0.4), ('NP', 0.04))
('V', 1.0)
('V', 1.0)
('NP', -3.2188758248682006)


ValueError: ignored

In [11]:
def CKYParseMarginalized(words, grammar, table):
  #part4
  return table, tree, probablility

In [12]:
#main code segment
#part 2 implementation
words = ["British", "left", "waffles", "on", "Falklands"]
grammar = {"British": ["NP", "JJ"], "left":["NP","VP"], "waffles":["NP", "VP"], "on": ["P"], "Falklands":["NP"],
           "non-terminal": [["S" , "NP", "VP"],["NP" ,  "JJ", "NP"],["VP", "VP", "NP"], ["VP", "VP", "PP"], ["PP",  "P", "NP"]]}
table = [[set() for j in range (len(words))] for i in range(len(words))] #list of sets?
#print (table)
parse_table = CKYParse(words, grammar, table)
print (tabulate(parse_table))



#part 3 implementation
words = ["astronomers", "saw", "stars", "with", "ears"]
grammar = {"non-terminal": [("S", "NP", "VP", 1.0), ("NP", "NP", "PP", 0.4), ("PP", "P", "NP", 1.0),("VP", "V", "NP", 0.7), ("VP", "VP", "PP", 0.3)],
                            "astronomers": [("NP", 0.4)] ,  "ears": [("NP", 0.18)], "saw": [("NP", 0.04), ("V", 1.0)], "with": [("P", 1.0)], 
                            "stars": [("NP", 0.18)], "telescopes": [("NP", 0.1)]}
table = [[set() for j in range (len(words))] for i in range(len(words))] 
parse_results = CKYParseWeighted(words, grammar, table)
print (tabulate(parse_results[0]))
print (parse_results[1])
print (parse_results[2])

#part 4 implementaiton
table = [[set() for j in range (len(words))] for i in range(len(words))] 
#parse_results = CKYParseMarginalized(words, grammar, table)
print (tabulate(parse_results[0]))
print (parse_results[1])
print (parse_results[2])



------------  ------------  ------------  -----  -----------
{'JJ', 'NP'}  {'S', 'NP'}   {'S'}         set()  {'S'}
set()         {'VP', 'NP'}  {'VP', 'S'}   set()  {'VP', 'S'}
set()         set()         {'VP', 'NP'}  set()  {'VP'}
set()         set()         set()         {'P'}  {'PP'}
set()         set()         set()         set()  {'NP'}
------------  ------------  ------------  -----  -----------
------  -----------  ------  -----  ------
{'NP'}  set()        {'S'}   set()  {'S'}
set()   {'V', 'NP'}  {'VP'}  set()  {'VP'}
set()   set()        {'NP'}  set()  {'NP'}
set()   set()        set()   {'P'}  {'PP'}
set()   set()        set()   set()  {'NP'}
------  -----------  ------  -----  ------
[]
[]
------  -----------  ------  -----  ------
{'NP'}  set()        {'S'}   set()  {'S'}
set()   {'V', 'NP'}  {'VP'}  set()  {'VP'}
set()   set()        {'NP'}  set()  {'NP'}
set()   set()        set()   {'P'}  {'PP'}
set()   set()        set()   set()  {'NP'}
------  -----------  ------  --